In [6]:
import sys
sys.path.insert(0, "..")
import numpy as np
import glob
import concurrent.futures as fs
from utils import misc
from multiprocessing import Pool
import time
import utils
from distributed import sharded_matrix



In [2]:
imagenet_train_files = glob.glob("/home/ubuntu/imagenet-train-conv5/*.npy")


In [25]:
imagenet_val_files = glob.glob("/home/ubuntu/imagenet-val-conv5/*.npy")

In [34]:
train_shape = (len(imagenet_train_files), np.load(imagenet_train_files[0]).ravel().shape[0])
val_shape = (len(imagenet_val_files), np.load(imagenet_val_files[0]).ravel().shape[0])


In [4]:
X = np.memmap("/dev/shm/X_train_conv_5.npy", mode="w+", shape=train_shape, dtype="float32")
y = np.memmap("/dev/shm/y_train_conv_5.npy", mode="w+", shape=len(imagenet_train_files), dtype="int")
del X
del y
all_classes = list(set(list(map(lambda x: x.split("/")[-1].split("_")[0], imagenet_train_files))))


In [35]:
X_val = np.memmap("/dev/shm/X_val_conv_5.npy", mode="w+", shape=val_shape, dtype="float32")
y_val = np.memmap("/dev/shm/y_val_conv_5.npy", mode="w+", shape=len(imagenet_val_files), dtype="int")


In [38]:
def load_into_mmap_train(lst, idxs):
    X = np.memmap("/dev/shm/X_train_conv_5.npy", mode="r+", shape=train_shape, dtype="float32")
    y = np.memmap("/dev/shm/y_train_conv_5.npy", mode="r+", shape=len(imagenet_train_files), dtype="int")
    good_indices = []
    for arr, idx in zip(lst, idxs):
        try:
            class_name = arr.split("/")[-1].split("_")[0]
            x = np.load(arr).ravel()
            X[idx] = x
            y[idx] = all_classes.index(class_name)
            good_indices.append(idx)
        except:
            continue
    return good_indices 

def load_into_mmap_val(lst, idxs):
    X = np.memmap("/dev/shm/X_val_conv_5.npy", mode="r+", shape=train_shape, dtype="float32")
    y = np.memmap("/dev/shm/y_val_conv_5.npy", mode="r+", shape=len(imagenet_val_files), dtype="int")
    good_indices = []
    for arr, idx in zip(lst, idxs):
        try:
            class_name = arr.split("/")[-1].split(".")[0]
            x = np.load(arr).ravel()
            X[idx] = x
            y[idx] = all_classes.index(class_name)
            good_indices.append(idx)
        except:
            continue
    return good_indices 


In [6]:
pool_size = 64

p = Pool(pool_size)
chunked_files = list(misc.chunk(imagenet_train_files, len(imagenet_train_files)//pool_size))
chunked_idx = list(misc.chunk(list(range(len(imagenet_train_files))), len(imagenet_train_files)//pool_size))
chunked_futures = []
for i in range(len(chunked_files)):
    print("Submitting job {0}".format(i))
    future = p.apply_async(load_into_mmap, (chunked_files[i], chunked_idx[i]))
    chunked_futures.append(future)
    
all_idxs = []
for f in chunked_futures:
    %time all_idxs.extend(f.get())
p.terminate()




Submitting job 0
Submitting job 1
Submitting job 2
Submitting job 3
Submitting job 4
Submitting job 5
Submitting job 6
Submitting job 7
Submitting job 8
Submitting job 9
Submitting job 10
Submitting job 11
Submitting job 12
Submitting job 13
Submitting job 14
Submitting job 15
Submitting job 16
Submitting job 17
Submitting job 18
Submitting job 19
Submitting job 20
Submitting job 21
Submitting job 22
Submitting job 23
Submitting job 24
Submitting job 25
Submitting job 26
Submitting job 27
Submitting job 28
Submitting job 29
Submitting job 30
Submitting job 31
Submitting job 32
Submitting job 33
Submitting job 34
Submitting job 35
Submitting job 36
Submitting job 37
Submitting job 38
Submitting job 39
Submitting job 40
Submitting job 41
Submitting job 42
Submitting job 43
Submitting job 44
Submitting job 45
Submitting job 46
Submitting job 47
Submitting job 48
Submitting job 49
Submitting job 50
Submitting job 51
Submitting job 52
Submitting job 53
Submitting job 54
Submitting job 55
Su

In [42]:
y_val[:100]

memmap([627, 955, 390, 450, 594, 107,  36, 364,  35, 531, 560, 785, 684,
       319,  68, 606, 572, 678,  96, 302,  67,  19, 526, 506, 934, 574,
       915, 247, 434, 250,  90,  89, 236, 213, 470, 269,  87, 846, 380,
       209, 838,  77, 518, 898, 877, 755, 558, 225, 976, 849, 248, 119,
       170, 288, 422, 954, 989, 381, 721, 315, 715, 752, 213,  97, 138,
       742, 923, 920, 163, 244, 144, 941, 570, 661, 727, 482, 208, 684,
       184, 727,  82, 990, 711,  81, 749, 812, 417, 957, 401, 203,  70,
       949,  51, 508, 892, 630, 521, 724,  10,   3])

In [39]:
pool_size = 64
p = Pool(pool_size)
chunked_files = list(misc.chunk(imagenet_val_files, len(imagenet_val_files)//pool_size))
chunked_idx = list(misc.chunk(list(range(len(imagenet_val_files))), len(imagenet_val_files)//pool_size))
chunked_futures = []
for i in range(len(chunked_files)):
    print("Submitting job {0}".format(i))
    future = p.apply_async(load_into_mmap_val, (chunked_files[i], chunked_idx[i]))
    chunked_futures.append(future)
    
all_idxs = []
for f in chunked_futures:
    %time all_idxs.extend(f.get())
p.terminate()

Submitting job 0
Submitting job 1
Submitting job 2
Submitting job 3
Submitting job 4
Submitting job 5
Submitting job 6
Submitting job 7
Submitting job 8
Submitting job 9
Submitting job 10
Submitting job 11
Submitting job 12
Submitting job 13
Submitting job 14
Submitting job 15
Submitting job 16
Submitting job 17
Submitting job 18
Submitting job 19
Submitting job 20
Submitting job 21
Submitting job 22
Submitting job 23
Submitting job 24
Submitting job 25
Submitting job 26
Submitting job 27
Submitting job 28
Submitting job 29
Submitting job 30
Submitting job 31
Submitting job 32
Submitting job 33
Submitting job 34
Submitting job 35
Submitting job 36
Submitting job 37
Submitting job 38
Submitting job 39
Submitting job 40
Submitting job 41
Submitting job 42
Submitting job 43
Submitting job 44
Submitting job 45
Submitting job 46
Submitting job 47
Submitting job 48
Submitting job 49
Submitting job 50
Submitting job 51
Submitting job 52
Submitting job 53
Submitting job 54
Submitting job 55
Su

In [36]:
np.load("/home/ubuntu/imagenet-train-conv6/n01667114_6371.npy")

IOError: Failed to interpret file '/home/ubuntu/imagenet-train-conv5/n01667114_6371.npy' as a pickle

In [44]:
X.shape

NameError: name 'X' is not defined

In [7]:
X = np.memmap("/dev/shm/X_train_conv_5.npy", mode="r", shape=train_shape, dtype="float32")


In [110]:
y = np.memmap("/dev/shm/y_train_conv_5.npy", mode="r", shape=len(imagenet_train_files), dtype="int")


IOError: [Errno 2] No such file or directory: '/dev/shm/y_train_conv_5.npy'

In [12]:
X = X.astype('float64')


In [13]:
%time XTX = X.T.dot(X)


CPU times: user 2h 16min 56s, sys: 16 s, total: 2h 17min 12s
Wall time: 2min 35s


In [14]:
y_enc = np.eye(1000)[y]


In [15]:
y_enc.shape


(1281167, 1000)

In [16]:
%time Xty = X.T.dot(y_enc)


CPU times: user 21min 23s, sys: 14min 4s, total: 35min 27s
Wall time: 38.3 s


NameError: name 'scipy' is not defined

In [27]:
imagenet_val_files[0].split("/")[-1].split(".")[0]


'n01629819'

In [43]:
X

memmap([[  0.        ,   0.        ,   6.66407204, ...,   0.        ,
          0.        ,   0.        ],
       [ 17.5424099 ,  24.19426537,  21.59212112, ...,   0.78290975,
          0.        ,  21.28173065],
       [  0.        ,   0.        ,   0.        , ...,  28.85005379,
          0.        ,  13.59846592],
       ..., 
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [ 10.10374928,   0.65914536,   9.46424484, ...,   0.        ,
         21.03636742,  16.77190208],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [24]:
len(all_idxs)

1281166

In [45]:
X_val.shape




(50000, 9216)

In [22]:
import scipy.linalg
from sklearn import metrics


In [19]:
model = scipy.linalg.solve(XTX, Xty, sym_pos=True)

In [68]:
y_pred = X.dot(model)

In [69]:
metrics.accuracy_score(np.argmax(y_pred, axis=1), y)

0.45343581281753276

In [47]:
metrics.accuracy_score(np.argmax(y_pred, axis=1), y_val)

0.31879999999999997

In [104]:
from utils import exputil
import opt
from opt import ls

def compute_top_1_and_5_acc(labels, pred_weights):
    top1 = ls.top_k_accuracy(labels, pred_weights, k=1)
    top5 = ls.top_k_accuracy(labels, pred_weights, k=5)
    return top1, top5

def run_experiment_linear_conv_5_reg_sweep(X_train, X_test, labels_train, labels_test, XtX, XtY, regs=[1e7, 1e8]): 
    solve_params_template = {"solver": "direct_linear"}
    feat_params = {
    'featurization_scheme': 'conv5',
    }
    diag_idx = np.diag_indices(XtX.shape[0])
    all_results = []
    for reg in regs:
        start = time.time()
        solve_params = solve_params_template.copy()
        solve_params["regularization"] = reg
        XtX[diag_idx] += reg
        try:
            model = scipy.linalg.solve(XtX, XtY, sym_pos=True)
        except:
            XtX[diag_idx] -= reg
            raise
        XtX[diag_idx] -= reg
        print("Done with solve..calculating metrics now")
        end = time.time()
        y_train_pred = X_train.dot(model)
        y_test_pred = X_test.dot(model)
        
        solve_time = end - start
        top_1_train, top_5_train = compute_top_1_and_5_acc(labels_train, y_train_pred)
        top_1_test, top_5_test = compute_top_1_and_5_acc(labels_test, y_test_pred)
        results = {}
        results["train_top_1"] = top_1_train.tolist()
        results["train_top_5"] = top_5_train.tolist()
        results["test_top_1"] = top_1_test.tolist()
        results["test_top_5"] = top_5_test.tolist()
        results["solve_time"] = solve_time
        print(results)
        all_results.append((solve_params, feat_params.copy(), results, "conv5 linear reg sweep"))
    return all_results


    

In [ ]:
exp_results = run_experiment_linear_conv_5_reg_sweep(X, X_val, y, y_val, XTX, Xty, regs=[1e-8, 1e-5, 1e-2, 1, 10, 1e3]) 

In [ ]:
a = exp_results[0][2]['train_top_5']

In [100]:
a.tolist()

0.7249492064656676

In [105]:

exp_results = run_experiment_linear_conv_5_reg_sweep(X, X_val, y, y_val, XTX, Xty, regs=[0,10,1e4,1e5,1e6,1e7,1e8,1e9,1e10])

Done with solve..calculating metrics now
{'train_top_1': 0.45343581281753276, 'test_top_5': 0.56204, 'train_top_5': 0.7249492064656676, 'test_top_1': 0.3188, 'solve_time': 2.674391031265259}
Done with solve..calculating metrics now
{'train_top_1': 0.45343581281753276, 'test_top_5': 0.56204, 'train_top_5': 0.7249492064656676, 'test_top_1': 0.3188, 'solve_time': 3.48016619682312}
Done with solve..calculating metrics now
{'train_top_1': 0.45340068859094873, 'test_top_5': 0.56204, 'train_top_5': 0.7249164238541892, 'test_top_1': 0.31886, 'solve_time': 2.64454984664917}
Done with solve..calculating metrics now
{'train_top_1': 0.4531134504713281, 'test_top_5': 0.56222, 'train_top_5': 0.7245644010499802, 'test_top_1': 0.31878, 'solve_time': 2.9693119525909424}
Done with solve..calculating metrics now
{'train_top_1': 0.4501739429754279, 'test_top_5': 0.56306, 'train_top_5': 0.7212556989057632, 'test_top_1': 0.31922, 'solve_time': 2.596450090408325}
Done with solve..calculating metrics now
{'tr

In [106]:
exp_results

[({'regularization': 0, 'solver': 'direct_linear'},
  {'featurization_scheme': 'conv5'},
  {'solve_time': 2.674391031265259,
   'test_top_1': 0.3188,
   'test_top_5': 0.56204,
   'train_top_1': 0.45343581281753276,
   'train_top_5': 0.7249492064656676},
  'conv5 linear reg sweep'),
 ({'regularization': 10, 'solver': 'direct_linear'},
  {'featurization_scheme': 'conv5'},
  {'solve_time': 3.48016619682312,
   'test_top_1': 0.3188,
   'test_top_5': 0.56204,
   'train_top_1': 0.45343581281753276,
   'train_top_5': 0.7249492064656676},
  'conv5 linear reg sweep'),
 ({'regularization': 10000.0, 'solver': 'direct_linear'},
  {'featurization_scheme': 'conv5'},
  {'solve_time': 2.64454984664917,
   'test_top_1': 0.31886,
   'test_top_5': 0.56204,
   'train_top_1': 0.45340068859094873,
   'train_top_5': 0.7249164238541892},
  'conv5 linear reg sweep'),
 ({'regularization': 100000.0, 'solver': 'direct_linear'},
  {'featurization_scheme': 'conv5'},
  {'solve_time': 2.9693119525909424,
   'test_top

In [108]:
reload(exputil)
for res in exp_results:
    exputil.save_results(*res, repo_path="/home/ubuntu/pictureweb/")

/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-26-02:47:07.197436.tar pictureweb --exclude-from <(find pictureweb -size +3M)'
bb17ebdbe2f4b4d5d8a53ddc39bb0fb2
/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-26-02:47:14.702468.tar pictureweb --exclude-from <(find pictureweb -size +3M)'
f55e850f1f3fd97068a88a6ce28fd947
/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-26-02:47:23.002899.tar pictureweb --exclude-from <(find pictureweb -size +3M)'
62f3210f61e1d59c6e10b4655457f77c
/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-26-02:47:32.189002.tar pictureweb --exclude-from <(find pictureweb -size +3M)'
612f7043643ef6ae062e60041973877f
/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-26-02:47:39.879507.tar pictureweb --exclude-from <(find pictureweb -size +3M)'
065708895c476643f51d62a16c32d381
/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-26-02:47:46.948671.tar pictureweb --exclude-from <(find pictureweb -

In [4]:
from distributed import sharded_matrix

ModuleNotFoundError: No module named 'distributed'

In [111]:
np.save("/dev/shm/y_train_conv5.npy", y)

In [2]:
import numpy as np
np.load("/dev/shm/y_train_conv5.npy")

array([204, 352, 515, ..., 803, 392, 930])

In [3]:
sharded_matrix.ShardedMatrix("X_train_conv5")

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-3-5ef05276e6a3>, line 1)